In [1]:
import pandas as pd
import numpy as np
import jieba
import pickle


In [2]:
from cleanse import separateUserContent,separateAllContent
from random import shuffle, randint
from gensim.models import word2vec
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense ,SimpleRNN, Embedding, Activation, LSTM, GRU,Bidirectional,GlobalMaxPooling1D,Dot,Lambda,Add,Conv1D
from keras.models import Model,  Sequential, load_model,model_from_json
from keras import backend as K
from keras.callbacks import ModelCheckpoint

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
tmp_tup_df =pd.read_pickle("validation_tup_list.pkl")
validation_tup_list =tmp_tup_df.values.tolist()
print(validation_tup_list[0])

['我的T102H一直重開機不能用 平板電腦説找筆電處理IMG_0264.JPG IMG_0265.JPG像平板那款已跑10分鐘了 一直說處理更新', '目前可能是在更新 建議您先等他跑完']


In [4]:
vec_model = word2vec.Word2Vec.load("word2vec.model")

In [5]:
f = open("word_index_dic.pkl","rb")
word_index = pickle.load(f)
f.close()

In [6]:
test_ans_num = 99
tmp_df =pd.read_pickle("candidate_100_ans.pkl")#讀取對每個問題的100個答案
test_ans_set =tmp_df.values.tolist()



In [7]:
print((test_ans_set[6][99]))

通常是5天不包含假日


In [8]:
def makeTestQuesFeatuers(vec_model,word_index):
    docs = []

   
            
    for i in range(len(validation_tup_list)):
        list1 = jieba.cut(validation_tup_list[i][0], cut_all = False)
    
        docs.append(list(list1))

   
   

    encoded_docs = []
    for list10 in docs:
        list_temp = []
        for word in list10:
            if word in vec_model.wv.vocab:
                list_temp.append(word_index[word])
        encoded_docs.append(list_temp)

    #print(encoded_docs[0])
    max_length = 40
    padded_docs_list = pad_sequences(encoded_docs, maxlen = max_length, padding = 'post')

    padded_docs = np.array(padded_docs_list)

    return padded_docs

def makeTestAnsFeatures(vec_model,word_index, test_num):
    docs = []

   
            
    for i in range(len(test_ans_set)):
        list1 = jieba.cut(test_ans_set[i][test_num], cut_all = False)
    
        docs.append(list(list1))

   
   

    encoded_docs = []
    for list10 in docs:
        list_temp = []
        for word in list10:
            if word in vec_model.wv.vocab:
                list_temp.append(word_index[word])
        encoded_docs.append(list_temp)

    #print(encoded_docs[0])
    max_length = 40
    padded_docs_list = pad_sequences(encoded_docs, maxlen = max_length, padding = 'post')

    padded_docs = np.array(padded_docs_list)

    return padded_docs


In [9]:
test_q_fea = makeTestQuesFeatuers(vec_model, word_index)#產生出的矩陣是（ ,40)
test_a_fea = []
for i in range(test_ans_num+1): #有100個答案的FEATURE 產生出的矩陣是（100, , 40)有100組答案每一組答案是(,40)
    tmp_list = makeTestAnsFeatures(vec_model,word_index,i)
    test_a_fea.append(tmp_list)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.471 seconds.
Prefix dict has been built succesfully.


In [10]:
print(len(test_a_fea))

100


In [11]:
json_file = open("model_margin_0.1_epo_10_2.json", 'r')#讀取MODEL
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

In [12]:
model.load_weights(filepath = "weights_best_2.hdf5")


In [13]:
ans = model.predict([test_q_fea,test_a_fea[0],test_a_fea[0]])#先預測第一句答案[RESULT , COS_POS COS_NEG因為要CONCAT
ans_cos_np = np.array(ans[1]).reshape((len(ans[1]),1))#從 (1,) to ( , 1)
#ans_neg_np = np.array(ans[2]).reshape((len(ans[1]),1))

In [ ]:
for i in range(1,test_ans_num+1):#predict 剩下的 大約15分
    ans = model.predict([test_q_fea,test_a_fea[i],test_a_fea[i]])
    tmp_cos_np = np.array(ans[1]).reshape((len(ans[1]),1))
    ans_cos_np = np.concatenate((ans_cos_np,tmp_cos_np),axis = 1)#把答案串起來 ( ,100)
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94


In [ ]:
true_ans_np = ans_cos_np 

In [ ]:
'''true_ans = []
for i in range(len(true_ans_np)):#把最大的位置記下來 
    max_num = true_ans_np[i][0]
    max_ind = 0
    for j in range(1,test_ans_num+1):
        if(true_ans_np[i][j] > max_num):
            max_ind = j
            max_num = true_ans_np[i][j]
    true_ans.append(max_ind)'''

In [ ]:
sorted_candidate_ans = []
for i in range(len(true_ans_np)):
    temp_list = []
    for j in range(0, test_ans_num+1):
        temp_list.append((test_ans_set[i][j],true_ans_np[i][j]))
    temp_list.sort(key=lambda tup: tup[1], reverse = True)
    sorted_candidate_ans.append(temp_list)
    

In [ ]:
print(test_ans_set[5][99] + " " + str(true_ans_np[5][99]))
sorted_candidate_ans[5][10][1]

In [ ]:
sum = 0
for i in range(len(sorted_candidate_ans)):
    if sorted_candidate_ans[i][0][1] == true_ans_np[i][99]:#最大的跟真實的答案一樣
        sum += 1
accuracy = sum / len(sorted_candidate_ans)
print(len(true_ans_np))
print(accuracy)

In [ ]:
sum = 0
for i in range(len(sorted_candidate_ans)):
    if sorted_candidate_ans[i][4][1] <= true_ans_np[i][99]:#真實的答案在第五明之前
        sum += 1
accuracy = sum / len(sorted_candidate_ans)
print(len(sorted_candidate_ans))
print(accuracy)

In [ ]:
sum = 0
for i in range(len(sorted_candidate_ans)):
    if sorted_candidate_ans[i][9][1] <= true_ans_np[i][99]:
        sum += 1
accuracy = sum / len(sorted_candidate_ans)
print(len(sorted_candidate_ans))
print(accuracy)

In [ ]:
'''sum = 0
for i in true_ans:#正確答案是在最後一個
    if i == 99:
        sum+=1
accuracy = sum / len(true_ans)
print(accuracy)'''

In [ ]:
with open("Output_test2.txt", "w") as text_file:
    for i in range(0, 10):
        text_file.write("Question %d : %s\n" % (i+1, validation_tup_list[i][0]))
        text_file.write("True answer %d : %s   %f \n" % (i+1, validation_tup_list[i][1], true_ans_np[i][99]))
        text_file.write("Ans:  \n\n")
        for j in range(0,10):
            text_file.write("%d. %s %f\n" % (j, sorted_candidate_ans[i][j][0], sorted_candidate_ans[i][j][1]))
    
        text_file.write("===========================\n")